In [0]:
import sys
!{sys.executable} -m pip install sqlalchemy
!{sys.executable} -m pip install pymysql
!pip install surprise
!pip install recmetrics
!pip install funcsigs

In [0]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

db_connection_str = 'mysql+pymysql:// '
db_connection = create_engine(db_connection_str)

In [0]:
df = pd.read_sql('SELECT * FROM retail_uk_ecom_ecommerce LIMIT 10', con = db_connection)
df

,id,transaction_source,channel_name,channel_postcode,Transaction_Group,sales_date,weekday_number,time_of_sale,Demand_Date,Demand_Time,...,RFM_Score,Is_Employee,customer_age,customer_title,customer_country_code,is_email_present,footfall_hourly,hour,Supplier_ID,new_postcode
0,1,eCommerce,eCommerce,,Sales (Despatched),2018-01-02,2,,2017-12-28 00:00:00,19:40,...,9,,NA,Mrs,UK,1,NA,NA,202,TN21
1,2,eCommerce,eCommerce,,Sales (Despatched),2018-01-02,2,,2018-01-01 00:00:00,15:04,...,9,,NA,Prof,US,1,NA,NA,227,02478
2,3,eCommerce,eCommerce,,Sales (Despatched),2018-02-09,5,,2018-02-08 00:00:00,20:46,...,5,,NA,Mr,GB,1,NA,NA,234,PE7
3,4,eCommerce,eCommerce,,Sales (Despatched),2018-01-02,2,,2017-12-31 00:00:00,17:50,...,2,,NA,Mrs,ES,1,NA,NA,232,26004
4,5,eCommerce,eCommerce,,Sales (Despatched),2018-02-09,5,,2018-02-08 00:00:00,14:38,...,3,,NA,Ms,SG,1,NA,NA,55,509731
5,6,eCommerce,eCommerce,,Sales (Despatched),2018-02-09,5,,2018-02-08 00:00:00,14:38,...,3,,NA,Ms,SG,1,NA,NA,55,509731
6,7,eCommerce,eCommerce,,Sales (Despatched),2018-01-02,2,,2017-12-29 00:00:00,08:33,...,3,,NA,Mrs,GB,1,NA,NA,232,SS9
7,8,eCommerce,eCommerce,,Returns,2018-02-01,4,,2017-12-24 00:00:00,12:28,...,9,,NA,Miss,GB,1,NA,NA,54,WR12
8,9,eCommerce,eCommerce,,Sales (Despatched),2018-02-01,4,,2018-01-31 00:00:00,12:57,...,8,,NA,Mrs,GB,1,NA,NA,236,IP4
9,10,eCommerce,eCommerce,,Returns,2018-02-01,4,,2017-12-24 00:00:00,12:28,...,9,,NA,Miss,GB,1,NA,NA,54,WR12


In [0]:
from surprise import SVD
from surprise import KNNBasic
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from funcsigs import signature
from surprise.model_selection import KFold
import recmetrics
from collections import defaultdict
from surprise.model_selection import train_test_split

In [0]:
df = pd.read_sql('''SELECT SKU, customer_account,items_quantity
                    FROM retail_uk_ecom 
                    WHERE items_quantity > 0 AND Product_SubCategory_Desc <> "Admin" AND transaction_lines_count > 1 AND customer_account <> 0
                    LIMIT 10000''', con = db_connection)
df['items_quantity'].unique()
reader = Reader(rating_scale=(0,10))
data = Dataset.load_from_df(df[['customer_account', 'SKU', 'items_quantity']], reader)
trainset, testset = train_test_split(data, test_size=.25)

In [0]:
algo_normal = NormalPredictor()
algo_normal.fit(trainset)
predictions = algo_normal.test(testset)
print(accuracy.rmse(predictions))
print(accuracy.mse(predictions))

RMSE: 0.2904
0.29042279825004946
MSE: 0.0843
0.08434540174338892


In [0]:
# Use the famous SVD algorithm.
algo_svd = SVD()
algo_svd.fit(trainset)
predictions = algo_svd.test(testset)
print(accuracy.rmse(predictions))
print(accuracy.mse(predictions))

RMSE: 0.2124
0.21238933901307144
MSE: 0.0451
0.04510923132640939


In [0]:
algo_knn = KNNBasic()
algo_knn.fit(trainset)
predictions = algo_knn.test(testset)
print(accuracy.rmse(predictions))
print(accuracy.mse(predictions))

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.2217
0.2216805738347003
MSE: 0.0491
0.04914227681568201


In [0]:
trainset = data.build_full_trainset()
algo = SVD()
algo.fit(trainset)

In [0]:
testset = trainset.build_anti_testset()
testset = testset[1:1000]

In [0]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [0]:
predictions = algo.test(testset)

top_n = get_top_n(predictions, n=3)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(f'UserID: {uid, [iid for (iid, _) in user_ratings]}')

UserID: ('3820981', ['730853', '687973', '687980'])
UserID: ('3821229', ['737319', '737807', '688673'])
UserID: ('3824883', ['688666', '690393', '742993'])


In [0]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [0]:

    trainset, testset = train_test_split(data, test_size=.25)
    algo.fit(trainset)
    predictions = algo.test(testset)
    precisions, recalls = precision_recall_at_k(predictions, k=7, threshold=2)


    # Precision and recall can then be averaged over all users
    print(f'the precision rate is {sum(prec for prec in precisions.values()) / len(precisions)}')
    print(f'the recall rate is {sum(rec for rec in recalls.values()) / len(recalls)}')

the precision rate is 1.0
the recall rate is 0.9712746858168761
